In [1]:
import numpy as np # linear algebra
import dask.array as da
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import sys

import matplotlib.pyplot as plt 
import seaborn as sns

import warnings
warnings.simplefilter("ignore")
df_train = pd.read_parquet("train.parquet")
df_train["S_2"] = pd.to_datetime(df_train["S_2"])
df_train = df_train.drop(['S_2'],axis = 1)

In [2]:
df_train

,customer_ID,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.005492,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.006986,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.006527,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.008126,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.979333,14,0.020818,0.828199,0.003487,0.090743,0.0,0.025139,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.001498,0
5531447,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.984907,10,0.007209,0.812610,0.005904,0.079886,0.0,0.023691,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.008225,0
5531448,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.983019,15,0.013151,0.815422,0.003457,0.100503,0.0,0.012343,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.006773,0
5531449,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.969861,15,0.009855,1.003541,0.005117,0.101802,0.0,0.008578,NaN,...,-1,-1,-1,0,0,0.0,NaN,0,0.001168,0


In [3]:
df_train = df_train.groupby(["customer_ID"]).tail(1).set_index("customer_ID")
df_train

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,NaN,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.002970,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.0,0.005068,NaN,0.060646,...,-1,-1,-1,0,0,0.000000,NaN,0,0.003169,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.880875,0,0.004284,0.812649,0.006450,NaN,0.0,0.007196,NaN,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.000834,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.0,0.009937,NaN,0.046104,...,-1,-1,-1,0,0,0.000000,NaN,0,0.005560,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.871900,0,0.007679,0.815746,0.001247,NaN,0.0,0.005528,NaN,0.044671,...,-1,-1,-1,0,0,0.000000,NaN,0,0.006944,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,0.844229,15,0.028515,1.009866,0.001928,0.128707,0.0,0.005893,NaN,0.113053,...,-1,-1,-1,0,0,0.000000,NaN,0,0.003009,0
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,0.831279,1,0.292360,0.055656,0.006953,NaN,0.0,0.233078,NaN,0.134540,...,-1,-1,-1,0,0,0.000000,NaN,0,0.009230,0
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,0.800522,9,0.020563,1.007023,0.000957,0.066648,0.0,0.006314,NaN,0.049778,...,-1,-1,-1,0,0,0.000000,NaN,0,0.000340,0


In [4]:

df_train_labels = pd.read_csv("train_labels.csv")
df_train_labels["target"] = df_train_labels["target"].astype("int8")
df_train_labels


,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0
...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1


In [5]:
df_train = df_train.merge(df_train_labels, on="customer_ID", how='left')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458913 entries, 0 to 458912
Columns: 190 entries, customer_ID to target
dtypes: float32(93), int16(9), int8(87), object(1)
memory usage: 215.8+ MB


In [6]:
tmp = df_train.isna().sum().mul(100).div(len(df_train)).sort_values(ascending=False)

tmp

D_88      99.819792
D_110     99.198541
B_39      99.105495
D_73      99.076296
B_42      98.661620
            ...    
D_78       0.000000
D_79       0.000000
R_8        0.000000
R_9        0.000000
target     0.000000
Length: 190, dtype: float64

In [7]:
missingDF = pd.DataFrame(tmp).reset_index()
drop_cols = missingDF[missingDF[0]>75]["index"].values
print(drop_cols)

['D_88' 'D_110' 'B_39' 'D_73' 'B_42' 'D_134' 'B_29' 'D_76' 'D_132' 'D_42'
 'D_142']


In [8]:
df_train

,customer_ID,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.002970,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.0,0.005068,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.003169,0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.880875,0,0.004284,0.812649,0.006450,NaN,0.0,0.007196,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.000834,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.0,0.009937,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.005560,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.871900,0,0.007679,0.815746,0.001247,NaN,0.0,0.005528,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.006944,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0.844229,15,0.028515,1.009866,0.001928,0.128707,0.0,0.005893,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.003009,0,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.831279,1,0.292360,0.055656,0.006953,NaN,0.0,0.233078,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.009230,0,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.800522,9,0.020563,1.007023,0.000957,0.066648,0.0,0.006314,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.000340,0,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.754129,0,0.015838,0.714486,0.000993,0.408849,0.0,0.050048,NaN,...,-1,-1,1,0,0.949723,0.446255,1,0.002502,2,1


In [9]:
df_train = df_train[['S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_11', 'S_12', 'S_13', 'S_15',
       'S_16', 'S_22', 'S_23', 'S_24', 'S_25', 'S_27','R_1', 'R_2', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'R_10',
       'R_12', 'R_13', 'R_24', 'R_26', 'R_27','B_1', 'B_2', 'B_3', 'B_4', 'B_6', 'B_7', 'B_9', 'B_10', 'B_11', 'B_14',
       'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_22', 'B_23', 'B_25', 'B_33',
       'B_37', 'B_38', 'B_40','D_39', 'D_41', 'D_44', 'D_47', 'D_48', 'D_51', 'D_54', 'D_55', 'D_58',
       'D_61', 'D_62', 'D_70', 'D_74', 'D_75', 'D_78', 'D_112','P_2','P_3','P_4','target']]

In [10]:
numeric_data = df_train.select_dtypes(include=[np.number])
categorical_data = df_train.select_dtypes(exclude=[np.number])

numeric_data.shape[1]
categorical_data.shape[1]

0

In [11]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(numeric_data)
numeric_columns =df_train.select_dtypes(np.number).columns
df_train[numeric_columns] = df_train[numeric_columns].fillna(df_train[numeric_columns].mean())
df_train[numeric_columns] = df_train[numeric_columns].round(3)
df_train.isna().sum()

S_3       0
S_5       0
S_6       0
S_7       0
S_8       0
         ..
D_112     0
P_2       0
P_3       0
P_4       0
target    0
Length: 72, dtype: int64

In [12]:
#categorical_data
#cols = ["D_68", "B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126"]
#df_train[cols] = df_train[cols].apply(pd.to_numeric, errors='coerce')
#df_train.isna().sum()

In [13]:
#data = df_train.sample(frac=0.1, random_state=786)
##data_unseen = df_train.drop(data.index).reset_index(drop=True)
#data.reset_index(drop=True, inplace=True)
y = df_train['target']
X = df_train.drop(['target'],axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=26,stratify=y)

print("X_train Training Data Size :",X_train.shape[0])
print("X_test Testing Data Size   :",X_test.shape[0])




X_train Training Data Size : 367130
X_test Testing Data Size   : 91783


In [14]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(367130, 71)
(367130,)
(91783, 71)
(91783,)


In [15]:
y_train

47450     0
212797    0
113264    1
321463    0
457975    0
         ..
259083    0
138644    0
365784    0
195989    0
231234    0
Name: target, Length: 367130, dtype: int8

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
def run_exps(X_train: pd.DataFrame , y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame) -> pd.DataFrame:
    '''
    Lightweight script to test many models and find winners
:param X_train: training split
    :param y_train: training target vector
    :param X_test: test split
    :param y_test: test target vector
    :return: DataFrame of predictions
    '''
    
dfs = []
models = [
          ('LogReg', LogisticRegression()), 
          ('RF', RandomForestClassifier()),
          ('KNN', KNeighborsClassifier()),
 #         ('SVM', SVC()), 
          ('GNB', GaussianNB()),
          ('XGB', XGBClassifier())
            ]
results = []
names = []
Total_pred = []
scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
target_names = ['1', '0']
for name, model in models:
        
        kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=26)
        cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_train)
        Pred_list = y_pred.tolist()
        Total_pred.append(y_pred.tolist())
        print(name)
        #print(classification_report(y_test, y_pred, target_names=target_names))
results.append(cv_results)
#Total_pred = Total_pred.append(Pred_list)
names.append(name)
print(cv_results)
this_df = pd.DataFrame(cv_results)
this_df['model'] = name
dfs.append(this_df)
final = pd.concat(dfs, ignore_index=True)


LogReg


KeyboardInterrupt: 

# Ensemble Learning

In [ ]:
Total_predpercentile_list = pd.DataFrame(
    {'lst1Title': lst1,
     'lst2Title': lst2,
     'lst3Title': lst3
    })

In [ ]:
Pred_data = pd.DataFrame(Total_pred)
Pred_data =  Pred_data.transpose()  
Pred_data

In [17]:
X_train

,S_3,S_5,S_6,S_7,S_8,S_11,S_12,S_13,S_15,S_16,...,D_61,D_62,D_70,D_74,D_75,D_78,D_112,P_2,P_3,P_4
47450,0.229,0.004,1,0.222,0,12,0.191,0,8,0.002,...,0.070,0.188,0,0,0,0,1.000,0.986,0.788,0.0
212797,0.539,0.111,0,0.847,1511,17,0.173,524,6,0.006,...,0.700,0.011,0,1,2,0,0.010,0.286,0.350,0.0
113264,0.563,0.000,0,0.405,1021,19,0.189,0,6,0.009,...,0.804,0.050,3,6,6,0,1.000,0.428,0.614,0.0
321463,0.111,0.094,0,0.228,2096,15,0.319,435,5,0.007,...,0.278,0.230,0,0,0,0,1.000,0.802,0.683,0.0
457975,0.170,0.113,0,0.260,528,13,0.172,435,12,0.001,...,0.453,0.188,0,0,0,-1,0.015,0.672,0.608,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259083,0.233,0.001,0,0.267,772,8,0.803,0,7,0.005,...,0.631,0.230,1,7,8,0,1.000,0.866,0.888,0.0
138644,0.229,0.001,1,0.222,0,12,0.186,0,8,0.002,...,0.036,0.484,0,0,0,0,1.000,0.947,0.747,0.0
365784,0.193,0.062,0,0.217,528,16,0.356,524,9,0.001,...,0.930,0.023,0,2,2,0,0.008,0.391,0.619,0.0
195989,0.147,0.016,0,0.107,996,11,0.341,290,7,0.003,...,0.008,0.451,0,0,1,0,1.000,0.934,0.598,0.0


In [18]:
y_train

47450     0
212797    0
113264    1
321463    0
457975    0
         ..
259083    0
138644    0
365784    0
195989    0
231234    0
Name: target, Length: 367130, dtype: int8

In [19]:
X_test

,S_3,S_5,S_6,S_7,S_8,S_11,S_12,S_13,S_15,S_16,...,D_61,D_62,D_70,D_74,D_75,D_78,D_112,P_2,P_3,P_4
443373,0.208,0.038,0,0.380,1898,7,0.193,0,4,0.001,...,0.664,0.020,1,1,2,-1,1.000,0.522,0.735,0.958
93990,0.091,0.003,0,0.059,2096,14,0.185,435,5,0.009,...,0.453,0.188,0,0,0,-1,1.000,0.629,0.658,0.000
408833,0.164,0.010,0,0.314,1021,21,0.654,39,6,0.002,...,0.939,0.013,0,1,2,0,1.000,0.366,0.807,0.959
178771,0.145,0.167,0,0.313,2380,11,0.131,435,5,0.004,...,0.917,0.144,0,3,3,0,1.000,0.527,0.568,0.000
373152,0.162,0.003,1,0.109,1133,19,0.192,0,4,0.005,...,0.906,0.022,3,6,7,2,0.018,0.237,0.582,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313723,0.229,0.002,1,0.222,0,12,0.191,0,8,0.009,...,0.260,0.540,0,0,0,0,1.000,0.837,0.576,0.000
313005,0.103,0.111,0,0.070,2218,10,0.393,435,5,0.007,...,0.032,0.223,0,0,0,0,1.000,0.850,0.677,0.962
313597,0.399,0.128,0,0.612,528,17,0.733,524,8,0.002,...,0.168,0.009,0,0,0,0,0.003,0.306,0.427,0.962
353278,0.229,0.003,1,0.222,0,12,0.186,0,8,0.007,...,0.802,0.389,0,2,2,0,1.000,0.831,0.774,0.000


In [ ]:
from numpy import array
from sklearn.model_selection import KFold
# data sample

# prepare cross validation
kfold = KFold(4, True, 1)
# enumerate splits
for train, test in kfold.split():
	print('train: %s, test: %s' % (data[train], data[test]))

In [23]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
estimator_list = [ ('LogReg', LogisticRegression()),
#          ('RF', RandomForestClassifier()),
          ('KNN', KNeighborsClassifier()),
 #         ('SVM', SVC()),
            ('lgb',lgb.LGBMClassifier()),
          ('GNB', GaussianNB()),
          ('XGB', XGBClassifier())
                 ]

stack_model = StackingClassifier(estimators=estimator_list,final_estimator=RandomForestClassifier())
stack_model.fit(X_train,y_train)


StackingClassifier(estimators=[('LogReg', LogisticRegression()),
                               ('KNN', KNeighborsClassifier()),
                               ('lgb', LGBMClassifier()), ('GNB', GaussianNB()),
                               ('XGB',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None, gamma=Non...
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                         

In [25]:
import joblib
import pickle
filename = "Completed_model3.joblib"
joblib.dump(stack_model, filename)

print("Model_saved")

Model_saved


In [26]:
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)
print(result)

0.8899578353289825


In [27]:
y_train_pred = stack_model.predict(X_train)
y_test_pred = stack_model.predict(X_test)



In [28]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

stack_model_train_accuracy = accuracy_score(y_train,y_train_pred)
stack_model_train_f1 = f1_score(y_train,y_train_pred,average = 'weighted')

In [29]:
print(stack_model_train_accuracy)
print(stack_model_train_f1)


0.8976493340233704
0.8972814201531647


In [ ]:

X_train

In [ ]:
df["S_2"] = pd.to_datetime(df_train["S_2"])
df= df_train.drop(['S_2'],axis = 1)